In [2]:
import torch
from torchaudio.models import Conformer, RNNT, HuBERTPretrainModel
import models

In [3]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

def get_total_params(model):
    return sum(p.numel() for p in model.parameters())

In [5]:
wav2vec = models.Wav2Vec(models.Wav2Vec_PARAMS)
wav2vec.load_state_dict(torch.load("models/checkpoints/wav2vec2-base-960h.pt"))

94393245

In [ ]:
HuBERTPretrainModel

